In [1]:
import torch
torch.cuda.is_available()

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
from datasets import load_dataset
from datasets import Dataset
dataset = load_dataset("sms_spam")
dataset = dataset.rename_column("sms", "text")
dataset = dataset['train']

processed_text = dataset['text']
display(type(processed_text))
processed_text = [s.replace('\n', '') for s in processed_text]

dataset = dataset.remove_columns(["text"])
display(dataset)
dataset = dataset.add_column("text", processed_text)
display(dataset)
dataset = dataset.shuffle(seed=42).select([i for i in list(range(5500))])


train_ds = Dataset.from_dict(dataset[0:4500])
validation_ds = Dataset.from_dict(dataset[4500:5000])
test_ds = Dataset.from_dict(dataset[5000:5500])





Reusing dataset sms_spam (C:\Users\DELL\.cache\huggingface\datasets\sms_spam\plain_text\1.0.0\53f051d3b5f62d99d61792c91acefe4f1577ad3e4c216fb0ad39e30b9f20019c)
100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 494.09it/s]


list

Dataset({
    features: ['label'],
    num_rows: 5574
})

Dataset({
    features: ['label', 'text'],
    num_rows: 5574
})

Loading cached shuffled indices for dataset at C:\Users\DELL\.cache\huggingface\datasets\sms_spam\plain_text\1.0.0\53f051d3b5f62d99d61792c91acefe4f1577ad3e4c216fb0ad39e30b9f20019c\cache-1dabca71be75cbef.arrow


In [3]:
display(train_ds)
display(validation_ds)
display(test_ds)

Dataset({
    features: ['label', 'text'],
    num_rows: 4500
})

Dataset({
    features: ['label', 'text'],
    num_rows: 500
})

Dataset({
    features: ['label', 'text'],
    num_rows: 500
})

In [4]:
display(train_ds[0:5])

{'label': [1, 0, 0, 0, 0],
 'text': ['sports fans - get the latest sports news str* 2 ur mobile 1 wk FREE PLUS a FREE TONE Txt SPORT ON to 8007 www.getzed.co.uk 0870141701216+ norm 4txt/120p ',
  "It's justbeen overa week since we broke up and already our brains are going to mush!",
  'Not directly behind... Abt 4 rows behind ü...',
  'Haha, my legs and neck are killing me and my amigos are hoping to end the night with a burn, think I could swing by in like an hour?',
  'Me too baby! I promise to treat you well! I bet you will take good care of me...']}

In [5]:
display(validation_ds[0:5])

{'label': [0, 0, 1, 0, 1],
 'text': ['Sent me ur email id soon',
  'Lol no. Just trying to make your day a little more interesting',
  "Want 2 get laid tonight? Want real Dogging locations sent direct 2 ur Mob? Join the UK's largest Dogging Network by txting MOAN to 69888Nyt. ec2a. 31p.msg@150p",
  "I'm still pretty weak today .. Bad day ?",
  'Moby Pub Quiz.Win a £100 High Street prize if u know who the new Duchess of Cornwall will be? Txt her first name to 82277.unsub STOP £1.50 008704050406 SP']}

In [6]:
display(test_ds[0:5])

{'label': [0, 0, 0, 0, 1],
 'text': ['Yeah so basically any time next week you can get away from your mom &amp; get up before 3',
  "Haha... can... But i'm having dinner with my cousin...",
  'How is your schedule next week? I am out of town this weekend.',
  'Alright i have a new goal now',
  'Check Out Choose Your Babe Videos @ sms.shsex.netUN fgkslpoPW fgkslpo']}

In [7]:
from transformers import set_seed

set_seed(42)

In [8]:
#defining model:
from transformers import (AutoTokenizer, 
                          AutoModelForSequenceClassification, 
                          Trainer, EarlyStoppingCallback,
                          TrainingArguments)

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenize_func = lambda sentences: tokenizer(sentences['text'], 
                                            padding='max_length', truncation=True, max_length = 100)



train_dataset = train_ds.map(tokenize_func, batched=True)
val_dataset = validation_ds.map(tokenize_func, batched=True)
test_dataset = test_ds.map(tokenize_func, batched=True)

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Parameter 'function'=<function <lambda> at 0x000001503EDE7E50> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  9.11ba/s]
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceCla

In [9]:
import numpy as np
from datasets import load_metric
 
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels, average='macro')["f1"]
   return {"accuracy": accuracy, "f1": f1}


training_args = TrainingArguments(
    output_dir=r'C:\Users\DELL\Text_Augmentation\sms_spam_dataset_3runs\models\Original Model',          # output directory
    num_train_epochs=20,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.1,               # strength of weight decay
    
    logging_dir=r'C:\Users\DELL\Text_Augmentation\sms_spam_dataset_3runs\logs\Original Model',            # directory for storing logs
    save_total_limit = 1,
    load_best_model_at_end=True,
    #metric_for_best_model='f1',
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    save_strategy='epoch',
    gradient_accumulation_steps=2,
    learning_rate=2e-5,
)


trainer = Trainer(
    model=model, 
    args=training_args, 
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

In [10]:
#training:
trainer.train(ignore_keys_for_eval=['last_hidden_state', 'hidden_states', 'attentions'])
# We are going to get multiple loss values on each training step here


The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 4500
  Num Epochs = 20
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 2
  Total optimization steps = 1400


Epoch,Training Loss,Validation Loss,Accuracy,F1
0,0.596400,0.406169,0.882000,0.468650
1,0.198200,0.055561,0.990000,0.975801
2,0.048800,0.034847,0.992000,0.980496
3,0.026400,0.029471,0.992000,0.980783
4,0.019900,0.035409,0.992000,0.980783
5,0.013300,0.027651,0.992000,0.980783
6,0.006400,0.025673,0.992000,0.981061
7,0.005100,0.035063,0.992000,0.980783
8,0.000800,0.028136,0.994000,0.985481
9,0.000500,0.030604,0.994000,0.985481


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 32
Saving model checkpoint to C:\Users\DELL\Text_Augmentation\sms_spam_dataset_3runs\models\Original Model\checkpoint-70
Configuration saved in C:\Users\DELL\Text_Augmentation\sms_spam_dataset_3runs\models\Original Model\checkpoint-70\config.json
Model weights saved in C:\Users\DELL\Text_Augmentation\sms_spam_dataset_3runs\models\Original Model\checkpoint-70\pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 32
Saving model checkpoint to C:\Users\DELL\Text_Augmentation\sms_spam_dataset_3runs\models\Original Model\checkpoint-140
Configuration saved in C:\Users\DELL\Text_Augme

TrainOutput(global_step=700, training_loss=0.09158696155462946, metrics={'train_runtime': 158.1829, 'train_samples_per_second': 568.961, 'train_steps_per_second': 8.851, 'total_flos': 1164264246000000.0, 'train_loss': 0.09158696155462946, 'epoch': 9.99})

In [11]:
#save model:
trainer.save_state()
trainer.save_model()

Saving model checkpoint to C:\Users\DELL\Text_Augmentation\sms_spam_dataset_3runs\models\Original Model
Configuration saved in C:\Users\DELL\Text_Augmentation\sms_spam_dataset_3runs\models\Original Model\config.json
Model weights saved in C:\Users\DELL\Text_Augmentation\sms_spam_dataset_3runs\models\Original Model\pytorch_model.bin


In [12]:
#evaluate:
trainer.evaluate(test_dataset)

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 32


{'eval_loss': 0.03539975732564926,
 'eval_accuracy': 0.99,
 'eval_f1': 0.9768241696099971,
 'eval_runtime': 4.064,
 'eval_samples_per_second': 123.032,
 'eval_steps_per_second': 3.937,
 'epoch': 9.99}